## 預載資料

In [1]:
import requests
import pandas
import sqlite3 as lite
from PIL import Image
from json import JSONDecoder
#import cv2
import time,os
import numpy as np
import random
import sys,os
import configparser as ConfigParser
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
import os,shutil,time
import numpy as np  
from imgurpython import ImgurClient

ini_data=ConfigParser.ConfigParser()
ini_data.read('key.ini')

#facepp資訊
key = ini_data.get('facepp','key')
secret = ini_data.get('facepp','secret')
#facepp資訊

#imgur資訊
client_id = ini_data.get('imgur','client_id')
client_secret = ini_data.get('imgur','client_secret')
access_token = ini_data.get('imgur','access_token')
refresh_token = ini_data.get('imgur','refresh_token')
client = ImgurClient(client_id, client_secret, access_token, refresh_token)
#imgur資訊

## 創建faceset

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/create"
data = {"api_key": key, "api_secret": secret, 'display_name':'FacesStore',"outer_id":input('請輸入名子：')}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(response)
print(req_dict)

## 製做facetoken

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/detect"

name=[input('請輸入女優的名子：')]#人物名子
ps=[input('請輸入女優的附註：')]
filepath = input('請輸入女優的檔名：')+'.jpg'#人物檔案位置_一定要羅馬拼音
logo=[input('請輸入logo網址：')]


#######################先上傳判斷############################
decide=1#try跳開
count=0#失敗計數數量
while decide:
    time.sleep(1)
    try:
        data = {"api_key": key, "api_secret": secret}
        files = {"image_file": open(filepath, "rb")}
        response = requests.post(http_url, data=data, files=files)

        req_con = response.content.decode('utf-8')
        req_dict = JSONDecoder().decode(req_con)

        print(req_dict)
       
        decide=0
    except KeyError as e3:
        count=count+1
        print('field')
        if count==8:
            decide=0

## 寫入資料庫程序

In [ ]:
pinyin=[filepath[0:filepath.find('.jpg')]]
face_token=[req_dict['faces'][0]['face_token']]

av_data={'名子':name,'羅馬拼音':pinyin,'face_tokens':face_token,'附註':ps,'logo':logo}
write=pandas.DataFrame(av_data)

with lite.connect('av_data.sqlite') as db:
    write.to_sql('face_search',con=db,if_exists='append')
    
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

## 在faceset裡面新增臉

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/addface"

data = {"api_key": key, "api_secret": secret, "outer_id": "line_bot_av_girl","face_tokens":'b50e099035902daff0a90213bf2be061'}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)

print(response)
print(response.text)

## 在faceset搜索

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/search"

filepath = "test2.jpg"

data = {"api_key": key, "api_secret": secret, "outer_id": "line_bot_av_girl","return_result_count":5}
files = {"image_file": open(filepath, "rb")}
response = requests.post(http_url, data=data, files=files)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(req_dict)


#與資料庫名子匹配
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)
ans_name=read[read["face_tokens"]==req_dict['results'][0]['face_token']].iloc[0,2]    
if req_dict['results'][0]['confidence']>83:
    word="我覺得是"+ans_name+"啦\n"+"相似程度有%f"%req_dict['results'][0]['confidence']
else:
    word="找不到相似的人 但可能是"+ans_name+"\n"+"相似程度有%f"%req_dict['results'][0]['confidence']

## 搜尋全部的Faceset

In [ ]:
http_url='https://api-cn.faceplusplus.com/facepp/v3/faceset/getfacesets'

data={"api_key": key, "api_secret": secret}

response = requests.post(http_url, data=data)
req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(response)

In [ ]:
req_dict

## 取得faceset列表

In [12]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/getdetail"

data = {"api_key": key, "api_secret": secret, "outer_id":'line_bot_av_girl','start':int(input('請輸入起始數字：'))}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(response)
print('檔案總數：'+str(req_dict['face_count']))
print('回傳tokens數量：'+str(len(req_dict['face_tokens'])))



請輸入起始數字：1
<Response [200]>
檔案總數：661
回傳tokens數量：100


## 伺服器比對電腦器

In [13]:
av_girl=os.listdir('av_girl_data')

av_girl.remove('失敗紀錄等等資訊.xls')
av_girl.remove('Client')
av_girl.remove('sure')
av_girl.remove('not_sure')
av_girl.remove('備份')
av_girl.remove('修正錯誤之後的圖片資料(未加入)')

av_pic_num=0
av_name={}
for item in av_girl:
    av_name[item]=len(os.listdir('av_girl_data/'+item))
    av_pic_num+=len(os.listdir('av_girl_data/'+item))

count=0
check_face_tokens=[]
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)
for item_s in req_dict['face_tokens']:
    for item_c in read['face_tokens'].tolist():
        if item_s==item_c:
            #print('電腦的:'+item_c+'||伺服器的:'+item_s)
            count=count+1
            check_face_tokens+=[item_s]
print('電腦數量：'+str(len(read['face_tokens'].tolist()))+'伺服器數量：'+str(req_dict['face_count']))
print('照片實際數量：'+str(av_pic_num))
for item in check_face_tokens:
    req_dict['face_tokens'].remove(item)
print('計數次數：'+str(count))
for item in req_dict['face_tokens']:
    print('錯誤檔案：'+item)

電腦數量：661伺服器數量：661
照片實際數量：661
計數次數：100


## 刪除faceset

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/delete"

data = {"api_key": key, "api_secret": secret, "outer_id": input('請輸入要刪除的faceset名稱：'),"check_empty":0}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(req_dict)




## 刪除face

In [32]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/removeface"

data = {"api_key": key, "api_secret": secret, "face_tokens": input('請輸入facetoken：'),'outer_id':'line_bot_av_girl'}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(req_dict)




請輸入facetoken：81d30e2b47510732c8f9d1c03d0b2878
{'faceset_token': 'bfd20391a10e47c37f798ac1eeb89e77', 'face_removed': 1, 'time_used': 189, 'face_count': 589, 'request_id': '1530459378,b5e9888b-3fb8-4d6f-9e00-b6eea9bc5c4e', 'outer_id': 'line_bot_av_girl', 'failure_detail': []}


## 以雲端表單新增女優資料
使用email_python-to-excel@python-to-excel.iam.gserviceaccount.com

In [9]:
import math

import time

#畫圖程序
import matplotlib.pyplot as plt
import numpy as np
#畫圖程序

import ctypes
import pandas
import requests
from bs4 import BeautifulSoup

from datetime import datetime

from flask import Flask, request, abort

#圖片辨識流程by face++
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
#圖片辨識流程by face++

import os,shutil#資料移動相關

####excel插件
import sys
import time
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC
####excel插件

In [10]:
GDriveJSON = 'python to excel-0ec3b563dac7.json'
GSpreadSheet = 'AV女優資料庫'

try:
    scope = ['https://www.googleapis.com/auth/drive']#原本採用'https://spreadsheets.google.com/feeds' 後來更為以上網址
    g_key = SAC.from_json_keyfile_name(GDriveJSON, scope)
    gc = gspread.authorize(g_key)
    sh = gc.open('AV女優資料庫')

except Exception as ex:
    print('無法連線Google試算表', ex)
    sys.exit(1)

In [11]:
worksheet = sh.worksheet("AV女優建立")
exl_star=input('請輸入開頭：')
exl_end=input('請輸入結尾：')

請輸入開頭：184
請輸入結尾：297


In [12]:
excel_data=worksheet.range('B'+exl_star+':'+'D'+exl_end)

In [13]:
for_excel_count=worksheet.range('B'+exl_star+':'+'B'+exl_end)

In [14]:
len(for_excel_count)

114

In [15]:
excel_count=0
for item in for_excel_count:
    print(excel_data[0+excel_count].value+'_'+excel_data[1+excel_count].value+'_'+excel_data[2+excel_count].value)
    
    cli_name=excel_data[0+excel_count].value
    cli_av_file_name=excel_data[1+excel_count].value
    cli_web=excel_data[2+excel_count].value
    
    excel_count=excel_count+3

桃生しお梨_A_https://i.imgur.com/9UVJxmw.jpg
福田優子_A_https://i.imgur.com/3T7t6Ju.jpg
美谷朱里_A_https://i.imgur.com/B5UDREW.jpg
中条みつ希_a_https://i.imgur.com/d8iMXlN.png
阿由葉あみ_v_https://i.imgur.com/xo7sjgk.png
笹原美香_a_https://i.imgur.com/59Vw3dZ.png
初川みなみ_a_https://i.imgur.com/dpONF2J.png
九重かんな_v_https://i.imgur.com/SIz0g5i.png
一条みお_a_https://i.imgur.com/6wyKWaa.png
柏木もも_a_https://i.imgur.com/UUPt1nM.png
青山希愛_a_https://i.imgur.com/yqwILoO.png
涼川絢音_a_https://i.imgur.com/B4xseIj.png
杏西みすず_a_https://i.imgur.com/sLWaDPs.png
三ツ矢ゆかり_v_https://i.imgur.com/deokCO3.png
飯倉芽衣_a_https://i.imgur.com/cMk2i3Y.png
乙原サラ_a_https://i.imgur.com/YjYbOKT.png
高橋しょう子_v_https://i.imgur.com/mJaLfc3.png
松田美子_a_https://i.imgur.com/rOujiR2.jpg
宝田もなみ_a_https://i.imgur.com/yROQ56A.png
富田優衣_v_https://i.imgur.com/utGAllk.png
堤あみ_v_https://i.imgur.com/vrHOXn3.png
神谷充希_a_https://i.imgur.com/OtsuDXG.jpg
色紙るな_a_https://i.imgur.com/NOXWCfP.png
里美ゆりあ_v_https://i.imgur.com/TJZKM9I.png
沢井みらい_a_https://i.imgur.com/QyLyHN4.png
瞳リョウ_v_https:

In [16]:
excel_count=0
update_faceset_name=input('請輸入faceset名子：')
for item in for_excel_count:
    print(excel_data[0+excel_count].value+'_'+excel_data[1+excel_count].value+'_'+excel_data[2+excel_count].value)
    
    cli_name=excel_data[0+excel_count].value
    cli_av_file_name=excel_data[1+excel_count].value
    cli_web=excel_data[2+excel_count].value
    
    excel_count=excel_count+3

    ##初始化
    name=[cli_name]#人物名子
    ps=['']#人物附註
    av_file_name=cli_av_file_name+'.jpg'#人物檔名 一定要羅馬拼音
    ##初始化
    
    #製作女優圖片
    print('--取得excel雲端相片')
    response=requests.get(cli_web,stream=True)
    print('--照片獲取狀態：'+str(response.status_code))

    with open(av_file_name,'wb') as f:
        f.write(response.content)
    #製作女優圖片

    #######################將圖片上傳製作facetoken############################
    #在faceset裡面新增臉
    print('--facetoken製作')
    http_url = "https://api-cn.faceplusplus.com/facepp/v3/detect"
    
    decide=1#try跳開
    count=0#失敗計數數量
    while decide:
        time.sleep(1)
        try:
            data = {"api_key": key, "api_secret": secret}
            files = {"image_file": open(av_file_name, "rb")}
            response = requests.post(http_url, data=data, files=files)

            req_con = response.content.decode('utf-8')
            req_dict_facetoken = JSONDecoder().decode(req_con)
            
            face_token=[req_dict_facetoken['faces'][0]['face_token']]
            
            print('確認'+req_dict_facetoken['faces'][0]['face_token'])
            print('人臉數量'+str(len(req_dict_facetoken['faces'])))
            decide=0
            print('--成功')
        except KeyError as e3:
            count=count+1
            print('上傳失敗')
            if count==8:
                print('--上傳錯誤')
                decide=0
                while True:
                    None
    files['image_file'].close()
    ########################將圖片上傳製作facetoken############################       
    if len(req_dict_facetoken['faces'])==1:#判斷圖片是否只有一人
        #在faceset裡面新增臉
        print('--facetoken上傳')
        http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/addface"

        decide=1#try跳開
        count=0#失敗計數數量
        while decide:
            time.sleep(1)
            try:
                data = {"api_key": key, "api_secret": secret, "outer_id": update_faceset_name,"face_tokens":face_token[0]}

                print('確認'+face_token[0])

                response = requests.post(http_url, data=data)

                req_con = response.content.decode('utf-8')
                req_dict = JSONDecoder().decode(req_con)
                
                web_face_count=str(eval(response.text)["face_count"])

                decide=0
                print('--成功')
            except KeyError as e3:
                count=count+1
                print('失敗上傳')
                if count==8:
                    print('--上傳錯誤')
                    decide=0
                    while True:
                        None
                    
    #建立女優資料##################################
        #製作女優資料夾
        print('--女優資料製作')
        try:
            os.mkdir('av_girl_data/'+name[0])
        except FileExistsError as e:
            None
        #製作女優資料夾

        #製作女優圖片檔名排序
        dir_file=os.listdir('av_girl_data/'+name[0])
        file_name=[]
        for item in dir_file:
            file_name+=[int(item[item.find('_')+1:item.find('.jpg')])]
        if file_name==[]:
            file_name=[0]
        #製作女優圖片檔名排序

        #移動圖片
        file_end='av_girl_data/'+name[0]+'/'+av_file_name[0:av_file_name.find('.jpg')]+'_'+str(max(file_name)+1)+'.jpg'
        shutil.move(av_file_name,file_end)
        #移動圖片
        
        #紀錄資料庫程序
        pinyin=[av_file_name[0:av_file_name.find('.jpg')]]
        
        #logo檢查程序 如果有就增加 沒有就空白
        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from face_search',con = db)
            if sum(read["名子"]==name[0])!=0:
                logo=read[read["名子"]==name[0]].iloc[0,6]
            else:
                logo=''
        #logo檢查程序 如果有就增加 沒有就空白

        av_data={'名子':name,'羅馬拼音':pinyin,'face_tokens':face_token,'附註':ps,'檔名':[av_file_name[0:av_file_name.find('.jpg')]+'_'+str(max(file_name)+1)+'.jpg'],'logo':[logo]}
        write=pandas.DataFrame(av_data)
        #紀錄資料庫程序
        
        #寫入資料程序(成功上傳再寫入)
        with lite.connect('av_data.sqlite') as db:
            write.to_sql('face_search',con=db,if_exists='append')

        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from face_search',con = db)
            print('網路人臉數量:'+web_face_count)
            print('伺服器人臉數量:'+str(len(read['檔名'].tolist())))
        print('--成功'+file_end+'\n\n')
    #建立女優資料##################################

    else:
        #電腦告知錯誤人數
        print('圖片人數不是一人')
        os.remove(av_file_name)
        print('--已刪除\n')
    
    #避免伺服器當掉 休息
    time.sleep(3)
    
    
    

請輸入faceset名子：line_bot_av_girl
桃生しお梨_A_https://i.imgur.com/9UVJxmw.jpg
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認9fc860ba514c54b456bb37ac0f57cd4d
人臉數量1
--成功
--facetoken上傳
確認9fc860ba514c54b456bb37ac0f57cd4d
--成功
--女優資料製作
網路人臉數量:663
伺服器人臉數量:663
--成功av_girl_data/桃生しお梨/A_1.jpg


福田優子_A_https://i.imgur.com/3T7t6Ju.jpg
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認1b59378996f3e8920f8b01a306b14449
人臉數量1
--成功
--facetoken上傳
確認1b59378996f3e8920f8b01a306b14449
--成功
--女優資料製作
網路人臉數量:664
伺服器人臉數量:664
--成功av_girl_data/福田優子/A_1.jpg


美谷朱里_A_https://i.imgur.com/B5UDREW.jpg
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認86b4acfffa82a59e34f0630d4886943d
人臉數量1
--成功
--facetoken上傳
確認86b4acfffa82a59e34f0630d4886943d
--成功
--女優資料製作
網路人臉數量:665
伺服器人臉數量:665
--成功av_girl_data/美谷朱里/A_2.jpg


中条みつ希_a_https://i.imgur.com/d8iMXlN.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認fc22a1f43473a613815641a3fa0e5328
人臉數量1
--成功
--facetoken上傳
確認fc22a1f43473a613815641a3fa0e5328
--成功
--女優資料製作
網路人臉數量:666
伺服器人臉數量:666
--成功av_girl_data/中条みつ希/

確認3bea0d728acea33e944ea4d7357c5020
人臉數量1
--成功
--facetoken上傳
確認3bea0d728acea33e944ea4d7357c5020
--成功
--女優資料製作
網路人臉數量:696
伺服器人臉數量:696
--成功av_girl_data/枢木あおい/a_1.jpg


持田栞里_v_https://i.imgur.com/4tSz3kQ.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認fbac2f4da81f7f84aa36352acc468762
人臉數量1
--成功
--facetoken上傳
確認fbac2f4da81f7f84aa36352acc468762
--成功
--女優資料製作
網路人臉數量:697
伺服器人臉數量:697
--成功av_girl_data/持田栞里/v_1.jpg


姫川ゆうな_a_https://i.imgur.com/nIssKcP.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認8bc6d8f3f34fda2a6a4f8fb9433a801d
人臉數量1
--成功
--facetoken上傳
確認8bc6d8f3f34fda2a6a4f8fb9433a801d
--成功
--女優資料製作
網路人臉數量:698
伺服器人臉數量:698
--成功av_girl_data/姫川ゆうな/a_1.jpg


高丘大空_a_https://i.imgur.com/CWZIWeY.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認031272429be7e8bcdf41a8b7375f9732
人臉數量1
--成功
--facetoken上傳
確認031272429be7e8bcdf41a8b7375f9732
--成功
--女優資料製作
網路人臉數量:699
伺服器人臉數量:699
--成功av_girl_data/高丘大空/a_1.jpg


横川あいの_v_https://i.imgur.com/RGAjw5T.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認86aaf11c93688f1485

琴古ひまり_a_https://i.imgur.com/k00eXNi.jpg
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認624889acb3dd3ebc82f936beeb79c2a4
人臉數量1
--成功
--facetoken上傳
確認624889acb3dd3ebc82f936beeb79c2a4
--成功
--女優資料製作
網路人臉數量:730
伺服器人臉數量:730
--成功av_girl_data/琴古ひまり/a_1.jpg


初美りん_a_https://i.imgur.com/2cQYtNS.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認d2415f2cc660badeb7670873e9a30e75
人臉數量1
--成功
--facetoken上傳
確認d2415f2cc660badeb7670873e9a30e75
--成功
--女優資料製作
網路人臉數量:731
伺服器人臉數量:731
--成功av_girl_data/初美りん/a_1.jpg


七瀬いずみ_a_https://i.imgur.com/EvJO4Xn.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認a646ae4c2bb5c075673f776f9dff4343
人臉數量1
--成功
--facetoken上傳
確認a646ae4c2bb5c075673f776f9dff4343
--成功
--女優資料製作
網路人臉數量:732
伺服器人臉數量:732
--成功av_girl_data/七瀬いずみ/a_1.jpg


日高まい_a_https://i.imgur.com/S1ZAPw0.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認6fed11ecc9c2644d399f15b934498bcb
人臉數量1
--成功
--facetoken上傳
確認6fed11ecc9c2644d399f15b934498bcb
--成功
--女優資料製作
網路人臉數量:733
伺服器人臉數量:733
--成功av_girl_data/日高まい/a_1.jpg


あかねりこ_a_https://i.im

確認440471e4e847a9b83801b720eccca9a7
--成功
--女優資料製作
網路人臉數量:763
伺服器人臉數量:763
--成功av_girl_data/若宮未來/a_1.jpg


松岡來未_a_https://i.imgur.com/oGnz3YF.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認38f87444d4a05860e6f433c8ffc4a4a4
人臉數量1
--成功
--facetoken上傳
確認38f87444d4a05860e6f433c8ffc4a4a4
--成功
--女優資料製作
網路人臉數量:764
伺服器人臉數量:764
--成功av_girl_data/松岡來未/a_1.jpg


里美ゆりあ_a_https://i.imgur.com/jeD7mPo.png
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認4ac8904c8dffa5dea7b7f97209762374
人臉數量1
--成功
--facetoken上傳
確認4ac8904c8dffa5dea7b7f97209762374
--成功
--女優資料製作
網路人臉數量:765
伺服器人臉數量:765
--成功av_girl_data/里美ゆりあ/a_2.jpg


濱松愛季_a_https://i.imgur.com/xu3ddEb.jpg
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認dcb8a0334e5733b3773c53c7f390615a
人臉數量1
--成功
--facetoken上傳
確認dcb8a0334e5733b3773c53c7f390615a
--成功
--女優資料製作
網路人臉數量:766
伺服器人臉數量:766
--成功av_girl_data/濱松愛季/a_1.jpg


白石みくり_a_https://i.imgur.com/IwIS93j.jpg
--取得excel雲端相片
--照片獲取狀態：200
--facetoken製作
確認f4583f65ec127d1ae46ddbcc09c60e96
人臉數量1
--成功
--facetoken上傳
確認f4583f65ec127d1ae46

In [17]:
print('計數:'+str(excel_count))
print('excel長度:'+str(len(excel_data)))
print('比對是否成功:'+str(excel_count==len(excel_data)))

計數:342
excel長度:342
比對是否成功:True


## 女優logo產生器

In [ ]:
import math

from imgurpython import ImgurClient#上傳圖片程序
import time

#畫圖程序
import matplotlib.pyplot as plt
import numpy as np
#畫圖程序

import ctypes
import pandas
import requests
from bs4 import BeautifulSoup

from datetime import datetime

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError,LineBotApiError
)
from linebot.models import (
    ImageCarouselColumn,ImageCarouselTemplate,CarouselColumn,CarouselTemplate,ImageMessage,MessageEvent, TextMessage, TextSendMessage,LocationMessage,FollowEvent,PostbackEvent,ImageSendMessage,FollowEvent,TemplateSendMessage,ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)

#圖片辨識流程by face++
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
#圖片辨識流程by face++

import os,shutil#資料移動相關

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db) 
    av_name=list(set(read[read['logo']=='']['名子'].tolist()))  

In [ ]:
av_name

In [ ]:
#新增全部
web=''
for item in av_name:
    if web!='no':
        print('目前女優:'+item+'\n')
        print("{0}".format('https://www.google.com.tw/search?q='+item+'+dmm'+'&source=lnms&tbm=isch')+'\n')
        web=input('logo網址:')
        if web!='no':
            with lite.connect('av_data.sqlite') as db:
                db.cursor().execute('UPDATE face_search set logo = "'+web+'" where 名子="'+item+'"')


In [ ]:
#只單純新增一人
item=input('輸入女優:')
print('目前女優:'+item+'\n')
print("{0}".format('https://www.google.com.tw/search?q='+item+'&source=lnms&tbm=isch')+'\n')
web=input('logo網址:')
with lite.connect('av_data.sqlite') as db:
    db.cursor().execute('UPDATE face_search set logo = "'+web+'" where 名子="'+item+'"')

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
read

## 資料庫女優檢查器

In [ ]:
av_girl=os.listdir('av_girl_data')

av_girl.remove('失敗紀錄等等資訊.xls')
av_girl.remove('Client')
av_girl.remove('sure')
av_girl.remove('not_sure')
av_girl.remove('備份')
av_girl.remove('修正錯誤之後的圖片資料(未加入)')

av_name={}
for item in av_girl:
    av_name[item]=len(os.listdir('av_girl_data/'+item))
    
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)
    
sql_name=read.groupby(by=['名子'])  
sql_name=sql_name.size().to_dict()
count=1
for item in av_name:
    try:
        if av_name[item]==sql_name[item]:
            None
        else:
            print(item+'　數量錯誤')
            print('資料夾擁有：'+str(av_name[item]))
            print('資料庫擁有：'+str(sql_name[item]))
    except KeyError as e1:
        print(item+'：沒有這個資料庫')
        
print('資料夾數量：'+str(len(av_name))+'\n資料庫數量：'+str(len(sql_name)))

## 女優辨識數量抓取器

In [ ]:
av_girl=os.listdir('av_girl_data/not_sure')

In [ ]:
av_girl[0].split('_')[0]

In [ ]:
av_girl

In [ ]:
count=int(0)
name=[]
for item in av_girl:
    
    name+=[item.split('_')[0]]
    name
        
    count+=1

In [ ]:
name

In [ ]:
myset = set(name)
for item in myset:
    print(' %s 有%d次' %(item,name.count(item)))


## 資料庫重複資料刪除器

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
for item in read['face_tokens'].tolist():
    try:
        if sum((read['face_tokens']==item).tolist())>1:

            #刪除重複資料(全部)
            with lite.connect('av_data.sqlite') as db:
                db.cursor().execute('delete from face_search where face_tokens="'+item+'"')
            
            face_token=read[read['face_tokens']==item].iloc[0,1]
            name=read[read['face_tokens']==item].iloc[0,2]
            av_file_name=read[read['face_tokens']==item].iloc[0,3]
            pinyin=read[read['face_tokens']==item].iloc[0,4]
            ps=read[read['face_tokens']==item].iloc[0,5]
            logo=read[read['face_tokens']==item].iloc[0,6]

            av_data={'名子':[name],'羅馬拼音':[pinyin],'face_tokens':[face_token],'附註':[ps],'檔名':[av_file_name],'logo':[logo]}
            write=pandas.DataFrame(av_data)
            #將資料回存(一筆)
            with lite.connect('av_data.sqlite') as db:
                write.to_sql('face_search',con=db,if_exists='append')
    except KeyError as e:
        None

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
read

## 女優大量圖網站爬蟲

In [ ]:
res=requests.get('http://sumomo-ch.com/archives.html')
html_doc =res.text
soup = BeautifulSoup(html_doc, 'html.parser')
web_av=[]
count=list(range(0,len(soup.find_all('a'))))
for item in count:
    if soup.find_all('a')[item].text=='AV女優':
        web_av+=[{'人名':soup.find_all('a')[item-1].text,'網址':soup.find_all('a')[item-1].get('href')}]

In [ ]:
for item in web_av:
    print(item['人名'][0:item['人名'].find(' ')])

In [ ]:
for item in web_av:
    print('http://sumomo-ch.com/%s'%(item['網址']))

## logo從資料庫新增

In [17]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db) 
    av_name=list(set(read[read['logo']=='']['名子'].tolist()))  

In [18]:
av_name

[]

In [19]:
for item in av_name:
    #開啟照片
    img_name='av_girl_data/%s/%s'%(item,os.listdir('av_girl_data/%s'%item)[0])
    print(img_name)
    imageA = Image.open(img_name)
    imageA = imageA.convert('RGB')
    (widthA , heightA) = imageA.size
    #創建白底
    imageB = Image.new('RGB', (heightA,heightA), (255, 255, 255))
    imageB = imageB.convert('RGB')
    (widthB , heightB) = imageB.size
    #抓置中點
    pos_w=int((widthB-widthA)/2)
    pos_h=int((heightA-heightB)/2)
    #貼上圖片
    imageB.paste(imageA,(pos_w,pos_h))
    if heightA>=300:
        imageB=imageB.resize( (290, 290), Image.BILINEAR )
        imageB.save("已合成圖片.jpg")
    else:
        imageB.save("已合成圖片.jpg")

    imgur_updata=client.upload_from_path("已合成圖片.jpg", config=None, anon=True)
    time.sleep(5)

    web=imgur_updata['link']
    print(imgur_updata['link'])
    with lite.connect('av_data.sqlite') as db:
        db.cursor().execute('UPDATE face_search set logo = "'+web+'" where 名子="'+item+'"')